In [1]:
import json
import pandas as pd
import yfinance as yf
import datetime as dt
from pytickersymbols import PyTickerSymbols
import pandas_datareader.data as web

# stock_data = PyTickerSymbols()
# countries = stock_data.get_all_countries()
# indices = stock_data.get_all_indices()
# industries = stock_data.get_all_industries()

stock_data = PyTickerSymbols()
dow_jones = list(stock_data.get_stocks_by_index('DOW JONES'))

In [2]:
symbols = {'Name': [], 'Symbol': []}

for ticker in dow_jones:
    symbols['Name'].append(ticker['name'])
    if type(ticker['symbol']) == str:
        symbols['Symbol'].append(ticker['symbol'])
    else:
        symbols['Symbol'].append(ticker['symbol']['yahoo'])

In [3]:
dow = pd.DataFrame(symbols)
dow.set_index('Symbol', inplace=True)
dow

,Name
Symbol,
MMM,3M Company
AXP,American Express Company
AAPL,Apple Inc.
BA,The Boeing Company
CAT,Caterpillar Inc.
CVX,Chevron Corporation
CSCO,Cisco Systems Inc.
KO,The Coca-Cola Company
GS,The Goldman Sachs Group Inc.


In [4]:
dow_symbols = list(dow.index)
dow_symbols

['MMM',
 'AXP',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WMT',
 'WBA',
 'DIS',
 'AMGN',
 'HON',
 'CRM',
 'DOW']

In [5]:
div_data = {}
count = 0
min_div_pay_years = 15

dow_symbols = list(dow.index)

for symbol in dow_symbols:
    count += 1
    if (count % 100 == 0):
        print(count)
    #print(f"getting dividend data from {symbol}")
    annual_div = {}
    prices = yf.Ticker(symbol).history(period='max')
    dividends = prices[prices['Dividends'] > 0]
    if len(dividends) > 0:
        first_year = dividends.index[0].year
        last_year = dt.datetime.today().year
    else:
        continue

    # print(dividends)

    # get annual dividend sum from first year it paid out div
    for year in range(first_year, last_year):
        div_sum = dividends[dividends.index.year == year]['Dividends'].sum()
        annual_div[year] = div_sum
    # min # years
    # div_data[symbol][0] = annual dividend sum
    # div_data[symbol][1] = # years dividends were paid out

    # if len(annual_div) >= year_threshold:
    if 0 in list(annual_div.values()):
        continue
    if len(annual_div) > min_div_pay_years:
        div_data[symbol] = []
        additionals = {}
        additionals['consecutive_yrs'] = len(annual_div)
        div_data[symbol].append(annual_div)
        div_data[symbol].append(additionals)

In [6]:
for symbol in div_data.keys():
    start_year = list(div_data[symbol][0].keys())[0]
    last_year = dt.datetime.today().year - 1
    prev_five_yrs = last_year - 6
    prev_ten_yrs = last_year - 11
    print(f"working on {symbol} | start year = {start_year}")
    historical_div_yield = []
    rate_change = []
    for year in range(prev_five_yrs, last_year + 1):
        rate_change.append(
            100 * (div_data[symbol][0][year] - div_data[symbol][0][year - 1]) /
            div_data[symbol][0][year - 1])
    five_yrs_div_growth_avg = sum(rate_change) / len(rate_change)
    div_data[symbol][1]['5YrsDivGrowthAvg'] = five_yrs_div_growth_avg
    
    historical_div_yield = []
    rate_change = []
    for year in range(prev_ten_yrs, last_year + 1):
        rate_change.append(
            100 * (div_data[symbol][0][year] - div_data[symbol][0][year - 1]) /
            div_data[symbol][0][year - 1])
    ten_yrs_div_growth_avg = sum(rate_change) / len(rate_change)
    div_data[symbol][1]['10YrsDivGrowthAvg'] = ten_yrs_div_growth_avg

working on MMM | start year = 1970
working on AXP | start year = 1977
working on BA | start year = 1962
working on KO | start year = 1962
working on GS | start year = 1999
working on HD | start year = 1987
working on IBM | start year = 1962
working on INTC | start year = 1992
working on JNJ | start year = 1962
working on JPM | start year = 1984
working on MRK | start year = 1970
working on MSFT | start year = 2003
working on NKE | start year = 1987
working on PG | start year = 1962
working on TRV | start year = 1987
working on UNH | start year = 1990
working on VZ | start year = 1984
working on WMT | start year = 1974
working on WBA | start year = 1985
working on HON | start year = 1970


In [7]:
div_data

{'MMM': [{1970: 0.082,
   1971: 0.115752,
   1972: 0.120376,
   1973: 0.13125,
   1974: 0.1565,
   1975: 0.16875,
   1976: 0.181,
   1977: 0.2125,
   1978: 0.25,
   1979: 0.3,
   1980: 0.35,
   1981: 0.375,
   1982: 0.4,
   1983: 0.4125,
   1984: 0.425,
   1985: 0.4375,
   1986: 0.45,
   1987: 0.465,
   1988: 0.53,
   1989: 0.65,
   1990: 0.73,
   1991: 0.78,
   1992: 0.8,
   1993: 0.83,
   1994: 0.88,
   1995: 0.94,
   1996: 2.16,
   1997: 1.06,
   1998: 1.1,
   1999: 1.12,
   2000: 1.16,
   2001: 1.2,
   2002: 1.24,
   2003: 1.32,
   2004: 1.44,
   2005: 1.68,
   2006: 1.84,
   2007: 1.92,
   2008: 2.0,
   2009: 2.04,
   2010: 2.1,
   2011: 2.2,
   2012: 2.36,
   2013: 2.54,
   2014: 3.42,
   2015: 4.1,
   2016: 4.44,
   2017: 4.7,
   2018: 5.44,
   2019: 5.76,
   2020: 5.88},
  {'consecutive_yrs': 51,
   '5YrsDivGrowthAvg': 13.198230876467122,
   '10YrsDivGrowthAvg': 9.749211940379826}],
 'AXP': [{1977: 0.083682,
   1978: 0.099774,
   1979: 0.115868,
   1980: 0.128744,
   1981: 0.09

In [8]:
def _get_close_and_historical_div_yield(symbol, period):
    startDate = (dt.date.today() - dt.timedelta(days=(365*(period + 1)))
                 ).strftime("%Y-%m-%d")
    endDate = dt.date.today().strftime("%Y-%m-%d")
    price_data = web.DataReader(symbol, 'yahoo', startDate,
                                endDate)

    # compute 5 years average dividend yield
    start_year = price_data.index[0].year + 1
    last_year = dt.date.today().year - 1

    dy_list = []

    for year in range(start_year, last_year + 1):
        yearly_data = price_data['Close'][price_data.index.year == year]
        firstPrice = yearly_data.iloc[0]
        lastPrice = yearly_data.iloc[-1]
        yearly_avg_price = (firstPrice + lastPrice) / 2
        yearly_dividend_yield = div_data[symbol.upper()][0][year] / yearly_avg_price
        dy_list.append(yearly_dividend_yield)

    historical_avg_dy = round((sum(dy_list) / len(dy_list)) * 100, 2)
    # print(historical_avg_dy)

    close = price_data['Adj Close'].iloc[-1].round(2)
    # print(close)
    #return (close, historical_avg_dy)
    return historical_avg_dy

def calculate_current_div_yield(symbol):
    
    startDate = (dt.date.today() - dt.timedelta(days=5)
                 ).strftime("%Y-%m-%d")
    endDate = dt.date.today().strftime("%Y-%m-%d")
    price_data = web.DataReader(symbol, 'yahoo', startDate,
                                endDate)
    
    prev_year = dt.date.today().year - 1
    
    prev_annual_div = div_data[symbol][0][prev_year]
    
    return prev_annual_div / price_data['Adj Close'][-1]

In [9]:
for symbol in div_data.keys():
    #print(symbol)
    #print(_get_close_and_historical_div_yield(symbol, 10))
    ten_years_div_yield_avg = _get_close_and_historical_div_yield(symbol, 10)
    current_div_yield = calculate_current_div_yield(symbol) * 100
    div_data[symbol][1]['10YrsDivYieldAvg'] = ten_years_div_yield_avg
    div_data[symbol][1]['CurrentDivYield'] = current_div_yield

In [10]:
div_data

{'MMM': [{1970: 0.082,
   1971: 0.115752,
   1972: 0.120376,
   1973: 0.13125,
   1974: 0.1565,
   1975: 0.16875,
   1976: 0.181,
   1977: 0.2125,
   1978: 0.25,
   1979: 0.3,
   1980: 0.35,
   1981: 0.375,
   1982: 0.4,
   1983: 0.4125,
   1984: 0.425,
   1985: 0.4375,
   1986: 0.45,
   1987: 0.465,
   1988: 0.53,
   1989: 0.65,
   1990: 0.73,
   1991: 0.78,
   1992: 0.8,
   1993: 0.83,
   1994: 0.88,
   1995: 0.94,
   1996: 2.16,
   1997: 1.06,
   1998: 1.1,
   1999: 1.12,
   2000: 1.16,
   2001: 1.2,
   2002: 1.24,
   2003: 1.32,
   2004: 1.44,
   2005: 1.68,
   2006: 1.84,
   2007: 1.92,
   2008: 2.0,
   2009: 2.04,
   2010: 2.1,
   2011: 2.2,
   2012: 2.36,
   2013: 2.54,
   2014: 3.42,
   2015: 4.1,
   2016: 4.44,
   2017: 4.7,
   2018: 5.44,
   2019: 5.76,
   2020: 5.88},
  {'consecutive_yrs': 51,
   '5YrsDivGrowthAvg': 13.198230876467122,
   '10YrsDivGrowthAvg': 9.749211940379826,
   '10YrsDivYieldAvg': 2.63,
   'CurrentDivYield': 2.9958883390209743}],
 'AXP': [{1977: 0.083682,

In [11]:
df_data = {'Symbol':[], '5YrsDividendGrowthAvg':[], '10YrsDividendGrowthAvg':[], 'CurrentDivYield':[], '10YrsDivYieldAvg':[]}

for symbol in div_data.keys():
    df_data['Symbol'].append(symbol)
    df_data['5YrsDividendGrowthAvg'].append(div_data[symbol][1]['5YrsDivGrowthAvg'])
    df_data['10YrsDividendGrowthAvg'].append(div_data[symbol][1]['10YrsDivGrowthAvg'])
    df_data['CurrentDivYield'].append(div_data[symbol][1]['CurrentDivYield'])
    df_data['10YrsDivYieldAvg'].append(div_data[symbol][1]['10YrsDivYieldAvg'])

In [12]:
df = pd.DataFrame(df_data)

In [13]:
df.set_index('Symbol', inplace=True)

In [14]:
df['DividendGrowthAvgOverall'] = (df['5YrsDividendGrowthAvg'] + df['10YrsDividendGrowthAvg'])/2
df

,5YrsDividendGrowthAvg,10YrsDividendGrowthAvg,CurrentDivYield,10YrsDivYieldAvg,DividendGrowthAvgOverall
Symbol,,,,,
MMM,13.198231,9.749212,2.995888,2.63,11.473721
AXP,10.428430,7.632396,1.160750,1.42,9.030413
BA,12.975203,9.234633,0.803048,2.24,11.104918
KO,5.624242,6.642854,3.100775,3.07,6.133548
GS,13.919618,11.694138,1.470156,1.45,12.806878
HD,21.434669,17.593524,1.874297,2.29,19.514097
IBM,8.546533,10.931417,4.946433,3.33,9.738975
INTC,5.675142,7.767380,2.047146,2.96,6.721261
JNJ,6.330899,6.865168,2.497490,2.94,6.598033


In [15]:
df.sort_values(by='DividendGrowthAvgOverall', ascending=False, inplace=True)
df

,5YrsDividendGrowthAvg,10YrsDividendGrowthAvg,CurrentDivYield,10YrsDivYieldAvg,DividendGrowthAvgOverall
Symbol,,,,,
UNH,24.484722,127.899080,1.279487,1.56,76.191901
JPM,15.216517,28.427563,2.356638,2.52,21.822040
HD,21.434669,17.593524,1.874297,2.29,19.514097
NKE,12.805103,12.950660,0.757519,1.20,12.877881
GS,13.919618,11.694138,1.470156,1.45,12.806878
MSFT,11.641314,13.572047,0.810178,2.29,12.606681
HON,12.419438,10.969834,1.578261,2.17,11.694636
MMM,13.198231,9.749212,2.995888,2.63,11.473721
BA,12.975203,9.234633,0.803048,2.24,11.104918


In [16]:
buy_list = df[df['CurrentDivYield']>df['10YrsDivYieldAvg']]

In [17]:
buy_list.sort_values(by='CurrentDivYield', inplace=True, ascending=False)

<ipython-input-17-1610c1a838a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_list.sort_values(by='CurrentDivYield', inplace=True, ascending=False)


In [18]:
buy_list

,5YrsDividendGrowthAvg,10YrsDividendGrowthAvg,CurrentDivYield,10YrsDivYieldAvg,DividendGrowthAvgOverall
Symbol,,,,,
IBM,8.546533,10.931417,4.946433,3.33,9.738975
WBA,6.674596,13.574331,3.372270,2.39,10.124464
KO,5.624242,6.642854,3.100775,3.07,6.133548
MMM,13.198231,9.749212,2.995888,2.63,11.473721
GS,13.919618,11.694138,1.470156,1.45,12.806878
